In [ ]:
%matplotlib inline
import sys, os, time, gc, click, logging, pprint
from os.path import expanduser


In [ ]:
%cd ..

In [ ]:
user_home = expanduser("~")
neuron_home = f'{user_home}/bin/nrnpython/lib/python'
model_home = f'{user_home}/src/model'
dentate_home = f'{model_home}/dentate' 
template_paths=f'{dentate_home}/templates'
config_prefix=f'{dentate_home}/config'
dataset_prefix='/media/igr/d865f900-7fcd-45c7-a7a7-bd2a7391bc40/Data/DG'

In [ ]:
import yaml
# creates 1) a yaml with synaptic parameters that have been optimized for selectivity
# 2) yaml with the same set of parameters as 1) but structured weight scaling factor set to 0
# format is { population : { gid: parameter list }}
# where parameter has the format: 
# postsynaptic pop., presynaptic populations, section type, receptor type, mechanism parameter, value
#
optimize_selectivity_params = \
{'GC': {111746: [['GC', ['BC', 'HCC', 'HC'], 'apical', 'GABA_A', 'weight', 0.20083801445743016],
                 ['GC', ['MC', 'ConMC'], 'apical', 'AMPA', ['weight', 'a'], 4.998258420749726],
                 ['GC', ['MPP', 'LPP'], 'apical', 'AMPA', ['weight', 'a'], 7.293403167741996],
                 ['GC', ['NGFC', 'MOPP'], 'apical', 'GABA_A', 'weight', 0.24140888269815286],
                 ['GC', 'AAC', 'ais', 'GABA_A', 'weight', 0.4534828996636781],
                 ['GC', 'BC', 'soma', 'GABA_A', 'weight', 0.412666789881585],
                 ['GC', 'NGFC', 'apical', 'GABA_B', 'weight', 0.005842658847846803]]}}
with open('selectivity_params_GC_111746_20210325.yaml', 'w') as f:
    yaml.dump(optimize_selectivity_params, f)
    
no_structured_weights_selectivity_params = \
{'GC': {111746: [['GC', ['BC', 'HCC', 'HC'], 'apical', 'GABA_A', 'weight', 0.20083801445743016],
                 ['GC', ['MC', 'ConMC'], 'apical', 'AMPA', ['weight', 'a'], 1.0],
                 ['GC', ['MPP', 'LPP'], 'apical', 'AMPA', ['weight', 'a'], 0.0],
                 ['GC', ['NGFC', 'MOPP'], 'apical', 'GABA_A', 'weight', 0.24140888269815286],
                 ['GC', 'AAC', 'ais', 'GABA_A', 'weight', 0.4534828996636781],
                 ['GC', 'BC', 'soma', 'GABA_A', 'weight', 0.412666789881585],
                 ['GC', 'NGFC', 'apical', 'GABA_B', 'weight', 0.005842658847846803]]}}
with open('no_structured_weights_selectivity_params_GC_111746_20210325.yaml', 'w') as f:
    yaml.dump(no_structured_weights_selectivity_params, f)

### Simulation with structured weights and synaptic paramters optimized for selectivity

In [ ]:
%run network_clamp.py go  -c Network_Clamp_GC_Exc_Sat_SLN_IN_Izh_extent.yaml \
         -p GC -g 111746  -t 9500 \
         --template-paths templates:{user_home}/src/model/DGC/Mateos-Aparicio2014 \
         --dataset-prefix {dataset_prefix} \
         --input-features-path {dataset_prefix}/Full_Scale_Control/DG_input_features_20200910_compressed.h5 \
         --input-features-namespaces 'Place Selectivity' \
         --input-features-namespaces 'Grid Selectivity' \
         --input-features-namespaces 'Constant Selectivity' \
         --arena-id A --trajectory-id Diag  --n-trials 1 \
         --recording-profile 'Network clamp default' \
         --use-coreneuron \
         --config-prefix config  --results-path results/netclamp \
         --results-file-id "demo" \
         --params-path selectivity_params_GC_111746_20210325.yaml

In [ ]:
%run scripts/plot_network_clamp.py -v -g 111746 --t-max=9500. \
-p results/netclamp/Single_GC_Exc_Sat_SLN_IN_Izh_extent_results_demo_selectivity_params_GC_111746_20210325.h5 \
--state-variable v --lowpass-plot-type=subplot \
-i MPP -i LPP -i MC -i ConMC -i GC \
--target-input-features-path={dataset_prefix}/Slice/dentatenet_Full_Scale_GC_Exc_Sat_SLN_extent_arena_margin_20210106a_compressed.h5 \
--config-file Network_Clamp_GC_Exc_Sat_SLN_IN_Izh_extent.yaml \
--target-input-features-arena-id=A \
--target-input-features-trajectory-id=Diag \
--font-size 18 \
--line-width 2


### Simulation without structured weights and synaptic paramters optimized for selectivity

In [ ]:
%run network_clamp.py go  -c Network_Clamp_GC_Exc_Sat_SLN_IN_Izh_extent.yaml \
         -p GC -g 111746  -t 9500 \
         --template-paths templates:{user_home}/src/model/DGC/Mateos-Aparicio2014 \
         --dataset-prefix {dataset_prefix} \
         --input-features-path {dataset_prefix}/Full_Scale_Control/DG_input_features_20200910_compressed.h5 \
         --input-features-namespaces 'Place Selectivity' \
         --input-features-namespaces 'Grid Selectivity' \
         --input-features-namespaces 'Constant Selectivity' \
         --arena-id A --trajectory-id Diag  --n-trials 1 \
         --recording-profile 'Network clamp default' \
         --use-coreneuron \
         --config-prefix config  --results-path results/netclamp \
         --results-file-id "demo" \
         --params-path no_structured_weights_selectivity_params_GC_111746_20210325.yaml

In [ ]:
%run scripts/plot_network_clamp.py -v -g 111746 --t-max=9500. \
-p results/netclamp/Single_GC_Exc_Sat_SLN_IN_Izh_extent_results_demo_no_structured_weights_selectivity_params_GC_111746_20210325.h5 \
--state-variable v --lowpass-plot-type=subplot \
-i MPP -i LPP -i MC -i ConMC -i GC \
--target-input-features-path={dataset_prefix}/Slice/dentatenet_Full_Scale_GC_Exc_Sat_SLN_extent_arena_margin_20210106a_compressed.h5 \
--config-file Network_Clamp_GC_Exc_Sat_SLN_IN_Izh_extent.yaml \
--target-input-features-arena-id=A \
--target-input-features-trajectory-id=Diag \
--font-size 18 \
--line-width 2
